In [97]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from typing import TypedDict
from dotenv import load_dotenv
load_dotenv()

True

In [98]:
llm = HuggingFaceEndpoint(
    repo_id='google/gemma-7b-it',
    task="text-generation"
)
model = ChatHuggingFace(llm=llm)

In [42]:

# llm = HuggingFaceEndpoint(
#     repo_id='meta-llama/Llama-3.1-8B-Instruct',
#     task="text-generation")
# model = ChatHuggingFace(llm=llm)

In [99]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [100]:
def create_outline(state: BlogState) -> BlogState:

    title = state["title"]

    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    state["outline"] = outline
    return state

In [101]:
def create_blog(state: BlogState) -> BlogState:

    title = state["title"]
    outline = state["outline"]

    prompt = f"Write a detailed blog on the title - '{title}' Using the following outline:\n{outline}"
    content = model.invoke(prompt).content

    state["content"] = content
    return state

In [102]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [ ]:
intial_state = {'title': 'Rise of AI in Pakistan'}

final_state = workflow.invoke(intial_state)

print(final_state)